In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Imports & Setting Configuration

In [ ]:
import re  
import numpy as np 
import pandas as pd 
from textblob import TextBlob

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from IPython.display import display, clear_output
pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows', 10000)

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english')) 

In [ ]:
# found on stackoverflow @ https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

##Import data & check it out

In [ ]:
#importing our youtube comments 
yt = pd.read_csv('/content/drive/MyDrive/data/3.26.youtube')
yt = yt.append(pd.read_csv('/content/drive/MyDrive/data/3.29.youtube'))
yt = yt.append(pd.read_csv('/content/drive/MyDrive/data/4.6.youtube'))
yt = yt.append(pd.read_csv('/content/drive/MyDrive/data/4.7.youtube'))
yt = yt.append(pd.read_csv('/content/drive/MyDrive/data/youtube_march20'))
yt = yt.append(pd.read_csv('/content/drive/MyDrive/data/youtube_march21'))
yt = yt.append(pd.read_csv('/content/drive/MyDrive/data/youtube_march28'))

In [ ]:
yt.head()

,Unnamed: 0,Unnamed: 0.1,video_id,text,likes,date,channel_id,viewer_rating,mentions,comment_id,Unnamed: 0.1.1
0,0,0.0,t59Ge4O70iM,Travel in 2020!!!😂😂😂,0.0,2020-09-13 16:52:14,UCqUn0dTF2TW80kVyPzsESxg,none,NaN,UgwT1aDIhDuG1o_x3VV4AaABAg,NaN
1,1,1.0,t59Ge4O70iM,If we really went to the moon. They would have...,1.0,2019-12-31 16:24:03,UCEhvCzLOt9oWeWq5xarhUtw,none,NaN,UgwshOzUbB13GGe8uIx4AaABAg,NaN
2,2,2.0,t59Ge4O70iM,So will this finally shut the flat earthers up...,1.0,2019-12-31 14:42:40,UCnLxgJ9qvXkbT1iqAhmMB0Q,none,NaN,UgxLXo627VME-wXGVnl4AaABAg,NaN
3,3,3.0,AhF44UT2AIk,My school sent me here ....,0.0,2021-03-09 09:54:23,UC8rZCL_bU2lfjNL_XwdwkkA,none,NaN,Ugx87ibygVo6-xDBRIB4AaABAg,NaN
4,4,4.0,AhF44UT2AIk,Whos here from mr bowles,0.0,2021-01-05 14:28:02,UC_48oH2diTx602x5T_dLOaw,none,NaN,Ugx0HRhg6p4jBsiyK294AaABAg,NaN


In [ ]:
yt.describe()

,Unnamed: 0,Unnamed: 0.1,likes,Unnamed: 0.1.1
count,482594.000000,226925.000000,482594.000000,146312.00000
mean,75431.468481,52394.775860,6.217614,73155.50000
std,59111.294792,44345.684815,132.582104,42236.78063
min,0.000000,0.000000,0.000000,0.00000
25%,20108.000000,14182.000000,0.000000,36577.75000
50%,64597.500000,35435.000000,0.000000,73155.50000
75%,124921.750000,89580.000000,1.000000,109733.25000
max,206025.000000,146311.000000,47771.000000,146311.00000


In [ ]:
# getting rid of duplicate rows 
yt = yt.drop_duplicates()
yt.reset_index(drop=True, inplace=True)
# starting a column to assign reposts to, so we have an equivalent 
# to retweets 
yt['repost_count'] = 0 
# checking how many times each comment occurs 
columns = {'index':'text', 'text':'count'}
txt_count = yt['text'].value_counts()
txt_count = txt_count.to_frame().reset_index().rename(columns=columns)
# editing our dataframe to add our repost column
for i in yt.index:
  text = yt.at[i, 'text']
  cnt = txt_count.loc[txt_count['text'] == text]
  if len(cnt) > 0: 
    cnt = cnt.reset_index(drop=True)['count'][0]
    yt.at[i, 'repost_count'] = cnt - 1

In [ ]:
# importing our twitter data 
twt = pd.read_csv('/content/drive/MyDrive/data/3.26.twitter', low_memory=False)
twt = twt.append(pd.read_csv('/content/drive/MyDrive/data/tweets_matt.csv',low_memory=False))

In [ ]:
twt.head()

In [ ]:
twt.describe()

In [ ]:
# renaming our columns so we can add our dataframes together 
yt = yt.rename(columns={'comment_id':'post_id', 
                        'likes':'favorite_count',
                        'channel_id':'user_id'})
twt = twt.rename(columns={'retweet_count':'repost_count'})
for i in [yt, twt]:
  i = i[['text', 'favorite_count', 'user_id',
         'mentions', 'repost_count', 'post_id']]

In [ ]:
# starting a count and an empty list for gathering new hashtags 
count = 0 
empty_list = []

In [ ]:
# look at the hashtags, see if there are any outliers to get rid of 
for item in twt.hashtag.value_counts().to_frame().reset_index()['index']:
    if count < 250:
        hashtags = str(item).split()
        for item in hashtags:
            empty_list.append(item)
            count = count + 1

# we don't want to count our original hashtags, so they're defined here
items = ['jaxa', 'esa', 'curiosityrover', 'areospace', 
         'internationalspacestation', 'JAXA', 'astronomy',
         'oppertunityrover', 'virgingalactic', 'universe', 
         'sls', 'Starship', 'climate', 'starship', 'virginorbit', 
         'nasa', 'cosmos', 'mars', 'falconheavy', 'NASA', 
         'futurism', 'starliner', 'iss', 'spacex', 'falcon9', 
         'nasa_app', 'roscosmos', 'Roscosmos', 'blueorigin',
         'ESA', 'spacetravel', 'artemis', 'marswebcam', 'starlink']

fresh_hashtags = []

for item in empty_list:
    if item not in items:
        fresh_hashtags.append(item)
        
print(set(fresh_hashtags))

{'Mars2020', 'JUNMYEON', 'Venera3', 'StanWorld', 'FalconHeavy', 'archaeology', 'perseverance', '솔라_빈센조ost_Adrenaline', 'MissionAlpha', 'Mars', 'SpaceX', 'KAI', 'mars2021', 'BecomeOneForIZONE', 'NFT', 'Doll', 'PerseveranceRover', 'Myanmar', 'Starlink', '또_만나슈_반가워요', 'OTD', '나의빛이되어준_진진_생일축하해', 'NowPlaying', 'EXO', 'XIUMIN_BeyondLIVE', 'MessierMarathon', 'Grammys', 'Perseverance', 'Webb', '카이', 'ISS', 'StrongAndCapable', '찬열', '시우민', 'didyouknow', 'BCU_RYS21', 'Perseverancerover', 'महर्षिदयानंद_का_अज्ञान', 'BackInPENSHOPPEtom6pm', '수호', 'SilkSonic', 'PiDay', 'IWD2021', 'Astronomy', '솔라', 'Navajo', 'HubbleFriday', 'SUHO', 'Astrophotography', 'Universe', 'Futurism', 'BanglaChaayeBJPModel', 'XIUMIN', 'InternationalWomensDay', 'ISSoverDaisenTottoriJP', 'XiuweetTimeWithYou', 'NASA_App', 'SolarAdrenalineOST', 'ExoMars', 'NFTs', 'InternationalSpaceStation', 'Baekhyun', 'Doyoung', '엑소', 'NASAWebb', 'Colchester', '백현', 'Mars2021', 'CountdownToMars', 'Space', 'NCT', 'CuriosityRover', 'SpecialPurpos

In [ ]:
# since we don't have the ends of some tweets due to the api cutoff
# and hashtags usually falll at the end, to make sure we have them all 
# we're going to check if they're in our sentences and if not add them
for i in range(len(twt)):
    if type(twt.at[i, 'hashtag']) == float:
        continue
    else:
      split_ = twt.at[i, 'hashtag']
      for item in split_:
        if type(twt.at[i, 'text']) == str:
           twt.at[i, 'text'] + ' ' + item

In [ ]:
# merging our dataframes 
df = twt.append(yt)
df = df.drop_duplicates(subset='post_id')

In [ ]:
# unsure how a handlful of strings ended up in our favorites but dropping
df.reset_index(drop=True, inplace=True)
for i in range(len(df)):
  if str(df.at[i, 'favorite_count']).isnumeric() != True:
    df.drop(index=i, inplace=True)

In [ ]:
# get rid of duplicate comments & retweets 
df = df.sort_values(by='favorite_count', ascending=False)
df = df.drop_duplicates(subset='text', keep='first')

In [ ]:
# much better 
df.head(10)

In [ ]:
df.describe()

In [ ]:
# making a dictionary containing every contraction and acronym to replace
replacements = {
                "ain't": "have not",
                "aren't": "are not",
                "can't": "cannot",
                "can't've": "cannot have",
                "'cause": "because",
                "could've": "could have",
                "couldn't": "could not",
                "couldn't've": "could not have",
                "didn't": "did not",
                "doesn't": "does not",
                "don't": "do not",
                "hadn't": "had not",
                "hadn't've": "had not have",
                "hasn't": "has not",
                "haven't": "have not",
                "he'd": "he had",
                "he'd've": "he would have",
                "he'll": "he shall / he will",
                "he'll've": "he shall have",
                "he's": "he is",
                "how'd": "how did",
                "how'd'y": "how do you",
                "how'll": "how will",
                "how's": "how is",
                "I'd": "I had",
                "I'd've": "I would have",
                "I'll": "I will",
                "I'll've": "I will have",
                "I'm": "I am",
                "I've": "I have",
                "isn't": "is not",
                "it'd": "it had",
                "it'd've": "it would have",
                "it'll": "it will",
                "it'll've": " it will have",
                "it's": "it is",
                "let's": "let us",
                "ma'am": "madam",
                "mayn't": "may not",
                "might've": "might have",
                "mightn't": "might not",
                "mightn't've": "might not have",
                "must've": "must have",
                "mustn't": "must not",
                "mustn't've": "must not have",
                "needn't": "need not",
                "needn't've": "need not have",
                "o'clock": "of the clock",
                "oughtn't": "ought not",
                "oughtn't've": "ought not have",
                "shan't": "shall not",
                "sha'n't": "shall not",
                "shan't've": "shall not have",
                "she'd": "she had ",
                "she'd've": "she would have",
                "she'll": "she will",
                "she'll've": "she will have",
                "she's": "she has ",
                "should've": "should have",
                "shouldn't": "should not",
                "shouldn't've": "should not have",
                "so've": "so have",
                "so's": "so as ",
                "that'd": "that would",
                "that'd've": "that would have",
                "that's": "that is",
                "there'd": "there had",
                "there'd've": "there would have",
                "there's": "there has",
                "they'd": "they had ",
                "they'd've": "they would have",
                "they'll": "they shall",
                "they'll've": "they shall have ",
                "they're": "they are",
                "they've": "they have",
                "to've": "to have",
                "wasn't": "was not",
                "we'd": "we had",
                "we'd've": "we would have",
                "we'll": "we will",
                "we'll've": "we will have",
                "we're": "we are",
                "we've": "we have",
                "weren't": "were not",
                "what'll": "what shall",
                "what'll've": "what shall have",
                "what're": "what are",
                "what's": "what has",
                "what've": "what have",
                "when's": "when has",
                "when've": "when have",
                "where'd": "where did",
                "where's": "where has",
                "where've": "where have",
                "who'll": "who shall",
                "who'll've": "who shall have",
                "who's": "who has",
                "who've": "who have",
                "why's": "why has",
                "why've": "why have",
                "will've": "will have",
                "won't": "will not",
                "won't've": "will not have",
                "would've": "would have",
                "wouldn't": "would not",
                "wouldn't've": "would not have",
                "y'all": "you all",
                "y'all'd": "you all would",
                "y'all'd've": "you all would have",
                "y'all're": "you all are",
                "y'all've": "you all have",
                "you'd": "you had",
                "you'd've": "you would have",
                "you'll": "you shall",
                "you'll've": "you shall have",
                "you're": "you are",
                "you've": "you have",
                " u " : " you ", 
                " w " : " with ",
                " poc ": " person of color ",
                " bpoc " : " black person of color ", 
                " bipoc ": " black indigenous people of color ",
                " lgbt " : " lesbian gay bisexual transgender ",
                " lgbtq ": " lesbian gay bisexual transgender queer ",
                " lgbtqi ": " lesbian gay bisexual" + 
                " transgender queer intersex", 
                " lgbtqia ": " lesbian gay bisexual" + 
                " transgender queer intersex asexual ", 
                " omg": " oh my god ",
                " lol ": " laughing out loud ", 
                " lmao ": " laughing my ass off ", 
                " lmk ":" let me know ",
                " icymi ": " in case you missed it ",
                " til ": " today i learned ",
                " omfg ": " oh my fucking god ", 
                " iirc ": " if i remember correctly ", 
                " ppl ": " people ",
                " ppls ": " peoples "
                " yts ": " white people "
                " yt ppl": " white people"
                " yt people": " white people"
                " yt girl": " white girl"
                " yt boy": " white boy"
                " yt men": " white men"
                " yt fragility ": " white fragility "
                " yt mf": " white motherfucker"
                " please ": " please "
                " rn ": " right now "
                " twt ": " tweet "
                " pov ": " point of view "
                " btw ": " by the way "
                " srsly ": " seriously "
                " fr ": " for real "
                " tho ": " though "
                " rlly ": " really "
                " rly ": " really "
                " ur ": " your "
                " ai ": " artificial intelligence "
                " ure ": " youre "
                " jfc ": " jesus fucking christ "
                " smh ": " shaking my head "
                " smfh ": " shaking my fucking head "
                " idiic ": " infinite diversity in infinite combinations "
                " llap ": " live long and propser "
                " idk ": " i dont know "
                " idc ": " I dont care "
                " ftw ": " for the win "
                " tbt ": " throw back tuesday "
                " n ": " and "
                " em ": " them "
                " oh em gee ": " oh my god "
                " ngl ": " not gonna lie "
                " b ": " be "
                " hmu ": " hit me up "
                " yt millennials ": " white millennials "
                " yt character ": " white character "
                " yt ": " youtube "
                " tf ": " the fuck "
                " af ": " as fuck "
                " asf ": " as fuck "
                " sm ": " so much "
                " irl ": " in real life "
                " aka ": " also known as "
                " jsyk ": " just so you know "
                " doe ": " though "
                " dey ": " they "
                " iwd": " international womens day "
                " pos ": " piece of shit "
                " mf ": " mother fucker "
                " boi": " boy "
                " w e ": " whatever "
                " wtf" : " what the fuck "
                " andamp " : " and "
                " tfw ": " that feeling when "
                " tl ": " timeline "
                " l8r ": " later "
                " im ": " i am "
                " id ": " i had "
                " youre ": " you are "
                " theyre ": " they are "
                " weve ": " we have "
                " ikr ": " i know right "
                " yall ": " you all "
                " isnt ": " is not "
                " dont ": " do not "
                " fyi ": " for your information "
                " cant ": " can not "
                " wasnt ": " was not "
                " tbh ": " to be honest "
                " andamp ": " and "
                " & " : " and "
                " yt " : "youtube"
                }

In [ ]:
# after thouroughly examining the data, we found a number of oddities that
# certainly dont belong, like slurs, pornographic content, dog photos
# we have them all defined so we can get rid of them here 
list_to_drop = [' supaspicyy ', ' snyder', ' avenger ', 'persona', 
         ' ghosting ', ' incel ', ' internetofblockchains ', 
         ' ot8 ', 'jar jar', ' pineapple on pizza ', 'side chick', 
         ' democrat ', ' prem ', ' remix ', ' fkn ', 'suppoawards',
         ' livestream ', 'hanamiiart', 'isolatiempo', 
         ' my coffee this morning ', ' exo kai ', 'demoncrats',
         ' coivd 19 pandemic', 'spotify', 'sailormoon', 'chyna',
         'check it out later dylan ', 'stonkmarket', 'oprah', 
         ' c 19', ' love your video', ' domestic au', ' sfw ', 
         ' shinee ', ' reggae ', ' undertale ', 'transmental', 
         'hogwarts', 'kakoadrawsstuff', 'ghhiphophubjoost', 
         ' english only ', 'streamtwice', ' avengers ', 'miningbc',
         ' lady gaga ', ' dnda ', 'dodgecoin', ' hello please ',
         ' freestyle ', ' bollywood ', ' destiny tl ', ' earth sign', 
         ' auschwitz ', ' an aries ', ' floof ', 'byunbaekyoong', 
         'getbtc', ' gif ', ' pythonbot ', 'star wars', 
         'downloaderbot', 'astrology', ' 1d ', 'bigdaddyxxi', 
         ' st  patricks day ', ' nipple ', ' dadbod ', ' junichiixo ',
         'dallasparkrec', 'minsung', ' jefferson starship ', 
         ' counterculture ', 'superjunior', 'page529', 
         'guardiansofthegala', ' acnh ', 'my gf', ' learn code ',
         'realtorlife', ' mnet ', 'justice league', ' caucasian',
         'donsbot', ' prince charles ', 'fakeyogurtfacts',
         'medicalcannabis', ' djs ', 'slytherin', ' brexit ', 
         ' buy 1 get 1 free', 'miteyuk', 'meekonmingmong', 
         ' pogg', ' tpc ', ' amc ', ' chu ', 'the8films',
         ' retard ', ' my cat ', ' ford motor company', ' au update',
         ' i just remembered ', ' icc ', 'tenfictions', 
         'idrawcharts', ' snyd ', 'space tv', 'kaliuchis', 
         ' choco pie ', 'australianart', 'bytesofbooks', ' yeaji ', 
         ' hot wife ', ' cancel culture ', ' youtube show', 
         ' cbdhage ', 'paragliding', ' i love you', ' funsized ',
         'ladyboy', ' game', ' st patrick ', 'kirk', 'my mom', 
         ' doctorwho ', ' good friday ', 'got7', 'challengertask ',
         ' shared on twitter ', ' out of character ',
         ' fucked me up yo ', ' gaga ', 'artrabbit', ' norse ',
         'definitelyfollow', ' the pandemic ', 'potterheadposts',
         ' ellle em ', ' anarcho ', ' mcu ', 'universe is with me ',
         'shouldhaveknownitwasascam', ' 8d ', ' basketball ',
         ' on zoom ', 'illegal alien', ' top 10', ' piday ', 
         'ffvlyricsbot', 'serena williams', ' chauvin ', ' sk8 ', 
         ' leovil ', 'canon', 'lizard people', 'blender3d', ' selfie ',
         ' twink ', ' prank ', 'gundamartemis', ' get popcorn ', 
         ' cassie and artemis ', ' net developer ', ' eddieizzard ',
         'southern  border', 'babyboy', 'canadanewswire', ' alchemy ',
         ' offseason ', ' au sommet ', ' game of thrones ', ' 9 11 ', 
         'lavigne', 'forsale', 'vanlife', ' blm ', ' webtoon ', 
         'hey fellas', ' good morning ', ' mac app ', 'tusculumsports', 
         'streetartsheff', 'italianostyle', 'suppoarea',  
         'dogagency', 'crazyforjoonie', ' handsome ', 'jaemjen', 
         ' new challenger appro', ' pound dumbbell ', ' ps4 ', 
         ' crossover ', 'hobbyconnector', ' klingon', 
         ' you dont know me ','africafactdaily',
         ' international womens day ', 'euchre', ' au where ', 
         'we kissed', 'geminis ',
         ' sk ', ' 911 ', ' discord servers ', 'brb', 'minimalisthyuk',
         ' starship  au ', ' ftp ', 'cravitystarship', 'wheephoric', 
         ' new virtual fieldtrip ', ' antarctic survey ', 
         ' st first contact', 'plandemic', ' horse teams ', 
         ' seminar ', ' doggg ', ' tony hawk ', ' follows today ', 
         ' como ', ' tag 10 ', ' leo sun ', 'nintendoswitch', 
         'spacecoin', 'lawlietslolita', 'onlinecasino', ' patreon ',
         ' senator ', ' baby momma ', 'bangtwicedaddyyungjoc', 
         'ariana grande', 'actualidadspacex', 'fandom', 'artpal', 
         'notamoviebuff', 'tony the tiger', ' pokemon', ' movie ', 
         ' sblv ', 'elysium', 'monchpublishing', 'witchinnreadin', 
         'trigger warning', ' otp ', 'happybirthday', 'hyeown',
         'domainnameforsale', ' podcast ', ' $btc ', ' concert ', 
         ' moonbin ', 'no other grape', ' drink and a snack ', 
         ' meghan markle ', 'north korea', ' albums ', 'sailorfemme',
         ' boba ', ' followed me ', ' toe suck', 'izone', 
         'planetfitness', ' sailor saturn ', ' waluigi ', 'vlogger',
         'shaun vid', ' batman ', '99memes', 'walmartemployees', 
         ' drag race', ' codenewbies ', 'ps5s', ' kike ', 'sex appeal',
         ' amazons bestselling ', 'ontariohealthc', 'yujin',
         'funnyanimalbook', 'tsla', ' plot hole', ' bittorrent ',
         ' giveway ', 'giveways', ' bucs ', ' stock ', 'gamingwithlance',
         ' skywalker ', ' jedi ', ' endgame ', ' wandavision ',
         ' girlgamer', ' celeste ', 'philzabot', 'includesartemismiss',
         'taylor swift', ' my girlfriend ', ' gogh ', 'lyricbot',
         ' stfu ', 'coinswitchkuber', ' nfts ', 'orbitalpodcast', 
         ' gypsy ', ' sherlock ', ' linkedin ', 'mythandfact', 
         'love the video', ' time travel au ', 'xiumin', ' boob ',
         'suppostreamers', ' kid cudi ', 'mrbeast', 'yourdailytds', 
         ' papa ', 'realtorneworkng', ' pinche ', 'newsparodypk',
         'mafia4rtemis', 'i have a crush', ' a taurus ', ' king kong ',
         'submissivetraining', ' suga ', ' greek ', 'thelegendof',
         ' yosemite ', 'new song', ' paddy', ' bestie ', ' vibe co',
         'mutualaidrequest', 'haikyuusalbot', 'grumpyexpolice', 
         ' the winner of the competition ', ' bien ', 'join us tonight', 
         'moonchild', 'ogleyuk', 'starship ent', 'busty', ' k pop ',
         ' 40k ', 'pockyoong', ' heckin ', 'nicki minaj', 
         ' chat tomorrow ', ' junhui ', ' danc', ' got me fucked up ',
         ' fan union ', ' ebooks ', 'mp4',  'dreamchaewon',
         'doe ever drive crazy fast night change', ' pi day ',
         'crackhore2', ' overwatch ', 'space bun', 'yeongyu', '30stm',
         ' italian mafia ', ' passed away yesterday ', 'twinsies', 
         'jongdae', 'seoho', 'pokmon', 'patrickstar', ' d r i n k  ', 
         'dprk', ' tron community ', 'sugacology', ' anime ', 
         ' ninetendo ', ' backup dance', ' otd ', ' bruno the goat', 
         ' drink and snack', ' rap ', 'club vibe', ' the titans ', 
         ' takki ', 'startrek lower decks', ' milf ', 'my song', 
         'exo member', 'world tour ', 'bakescookies4u', ' wonwoo ', 
         'portlandprotest', 'k followers', ' scandal ', ' nfl ',
         ' uni au ', 'newhollandbrew', ' ipad ', ' soloist', 'libtard',
         ' staan ', ' mars in gemini ', ' ggs ', ' woollim ', 
         'cutiesaturday', 'griffindor', ' feature on twitter ', 
         ' bidao ', ' blog pos', ' hyun ', ' ssbu ', ' my dog ', 
         ' girlsoftwitch ', ' pls rt', 'promoteamgamers', ' katalk', 
         ' tuning in tonight', ' childrens book', 'telepointo', 
         ' pinkmoontarot ', ' proofread my brothers ', ' eoy ', 
         ' graphic t ', ' his voice', ' mummy ', ' kanye ', 
         'lunarcrush', ' mew ', 'superiorartemis', ' trabajo',
         'kokobop', ' service animal ', ' incest ', ' skinny jean', 
         'ootd ', 'anamnesisstudio', ' cardi ', 'sethmeyers',
         'his pic', ' series e tournament ', ' monsta ', 'yedam', 
         'shemale', ' slashfic ', ' tienes ', ' joe manchin ', 
         ' ugly crying ', ' stans ', 'ebot ', ' gop challenger',
         'unovan', ' breaking news', ' de ', ' watermark ',
         ' exo vibe', ' painter ', 'spell check', ' maga ',
         'tokentizedtweets', ' fanfic ', ' dodgers ',
         'woman is killed', 'buffjjaeng', ' playlist ', 'jeonging',
         ' uber ', ' a pices ', 'joonsroom', ' hyde ', ' whore ', 
         ' senpai ', ' dj ', ' my office ', 'eeveelution', ' shran ', 
         ' youre so gorgeous ', ' baal', 'ujunghaeyo', ' baby girl ', 
         'rantarxamami', ' mommy ', ' data leak ', 'moondustyoon', 
         ' cr4vity ', 'hg2films', ' pfp ', ' vuelve ', 'for exo', 
         'huehue', ' snl ', 'lovato', ' channeling these vibes ', 
         'askabureson', ' nightvale', '911milfs', ' bois ', ' an esa ', 
         ' chanyeol ', ' exo perform ', 'ksjnluvbot', 'takeonthegame',
         ' me gusta ', ' my bf ', ' video game ', ' magic spell ', 
         ' spacex teq', ' sonic the hedge', ' terf ', 'suppodepartment', 
         'pokeballs', 'bearschoice2020', ' tits ', ' shadow the hedge', 
         ' air sign', 'pajeet26nazinazi', ' musician ', ' video recap ', 
         'youngboycharts', 'k iss', ' baby cakes ', ' qtum ', 
         ' plot holes ', 'fanart', 'leagueoflegends', ' vtube', 
         'webanime', ' romluan warbird ', ' cecil hotel', ' pissbab',
         'desiraiart', ' t4t ', ' au we all need', ' mfs ', 
         ' influencer ', ' instagram live ', ' a new article in no ', 
         ' ranbo', 'personal space', 'marchofrobots', ' you won ', 
         ' furry ', ' water sign', ' arachno futurism ', ' covid', 
         ' dank ', ' ethererum ', 'seuljhi', ' desis vibe ',
         ' horror movie', ' precious mom ', 'famedcelebrity', 
         'incantationsoverwater', 'join us at', 'fuckboy', ' jongin ', 
         ' the god ', ' seokjin', ' follow train ', ' challenger pack ',
         ' scam call ', 'highschool au', 'rick and morty', 
         'asminternatnl','justinbieb',' p l a y ',
         'dashboardconfessional', ' lads ', 'ramtrucks', ' stanning ', 
         'bad attitude ruin ', 'ig live', ' oc ', ' ugg ', 'dabak',
         ' artemis twt ', ' smashbrosultimate ', ' gaming ',  
         ' jinhoon', ' mercedes benz ', ' champion show ', 
         ' yung ', 'whatsupgurgaon', ' misogynist ', ' rape ', 
         'minsungbingo', ' superhero ', 'download the ap', 'stockpro', 
         ' hey babe ', ' internationalwomensday ', ' spacexcountdown ',
         'jaechan', ' popcorn spell', 'donaldtrump', ' no context', 
         '6ix9inebtcpromo', 'bruno mars', ' mama ', ' fingering', 
         'immigrantsnews', ' burger king ', 'bangers only', ' bd ',
         ' sun moon rising ', ' fancam ', ' fan page', ' my toes ',
         ' facist ', ' saint ', 'seungcheols', 'toryincompetence',
         'exol dks', ' titty ', 'y00nggas', 'stoner', 'ilor mercury ',
         'happyeasygo', ' commie ', ' audition ', ' vader ',  
         'hufflepuff', 'smash ultimate', ' gme ', ' sextile ',
         'pelosi', ' fifa ', ' starlink ', 'dateless', 'love geraniums', 
         ' au fic ', 'evertonschool', ' edgy ', 'leaked photo', 
         'jewish space lazer', ' itunes ', 'my commissions', 
         ' football ',  'faggot', ' pandora ',' cishet ', 
         'jewish space laser', 'fanartfriday', 'bbhlyuv', 's day ', 
         ' sailor moon ', ' 42069 ', 'ohmygirl', ' hot 100 ', 
         ' delivery fee ', ' blockchain', ' follow back', ' bitch ',
         ' teleheath ', ' if there were two guys on ', 'butthole', 
         'yvesjeong', ' donghyuk', ' women are killed ', 'disneyplus',
         'icone', 'rebranded', 'mawandegraphics', 'happening now ',
         ' zaddy ', 'herecomesshawty', ' police corrupt', ' stimmy',
         'minecraft', 'toploadersports', ' followers please', ' rin ', 
         ' followforfollow ', ' nationalpublicdefenderday ',' zionist ',
         'transgenders', ' esta ', ' exokai', ' march is ', 'accent', 
         ' ya boi ', ' leto ', ' pos coin', ' to code ', 'kylieart3', 
         ' yoda ', 'breesunshinee', 'headcannon', ' whatsapp ', 
         ' spell weaver ', ' page au ', 'i made out with ', 'ease rt ', 
         ' lin manuel ', 'onlyfans', 'weyoun', ' registered esa ', 
         'vmineverytime', ' thumbnail ', ' stoner', 'itsshowtimena',
         ' fck', ' my holdings', ' epiosde ', ' instagram ', 
         'humanesociety', 'song', ' zhongchi ', ' blockchain ', 
         ' btob ', ' bet wa ', ' cinnamon tea ', ' vibrator ', ' bnb ',
         ' my dad', ' abt abuse ', 'sumcoin', ' hades ', 'lupefiasco',
         ' cum ', ' dm me ', ' rapist ', ' waifu ', ' cravity ', 
         'chrysler', ' fancon ', 'hyoshin', 'releaseofliquidators',
         ' iwd', 'chipsmutcorner', ' itsyourmoment ', 'mp3', 
         ' alexandria ocas', ' mirrorverse ', 'stanislav', ' jk ',
         'cygames', ' rookie group ', ' henna ', 'btsarmy', ' yolo ', 
         ' stan ', ' history month ', ' 35mm film', ' in exo', 
         ' super junior ', ' praise god', 'magnesium for anx',
         ' hi ', 'choose character', 'mailsport','tedcruz', 
         'sn11 almost ready to fly ', ' dc uni', ' of the week', 
         ' dnd ', ' woke mob ', ' grader', ' stonline', 
         ' an aquarius ', ' roblox ', ' set your alarm',
         ' cameo ', 'transgendered', 'sayhername',
         ' reality design shows ', 'genre', ' boy group ',
         'armyrotc', ' xunqi', ' tip4da4', ' shino ', 'exhibitabrewing', 
         'furnituregurus', ' koreab00 ', ' dls ', ' binance ', 
         'doggo', 'orcaowlart', 'jaldi', '  did lockdown ', 
         ' top track ', ' lola and maya ', ' top 5 ', ' f4f ', 
         ' music video ', 'bobsburgersfox', 'criminaltiktoks', 
         ' kai exo ', ' rts ', 'paramoremusicom', ' shit post ', 
         ' yedam ', ' realistic arm', ' side part', ' friendsofkrispy ',
         ' prime timeline ', 'cobra starship', ' std ', ' cat food ', 
         'philwademusic', ' an yujin ', ' disney ', ' sl dorr ',
         ' footbal ', ' dm me for info ', ' au contraire', ' comedy ',
         'btfoshizzle', ' by exo ', 'jumpoon', ' st artemis ', 
         'comedybangbang', ' cardassia ', ' spoilers ', ' babymomma', 
         ' when exo ', ' cj enm ', 'theestallion', ' what is grammar ', 
         ' besties ', ' your own vibe ', 'justice4tamilgenocide', 
         'mindsofanime', ' sountrack ', ' we are hiring ', 'lola bunny',
         ' bb0un ', ' click link', ' sucking d', ' wonho ', 'bananocoin', 
         ' exo history', ' nike ', ' yuehua ', 'kroll show', 
         ' artemis fowl ', 'gayshit', 'antiasian', ' i woke up ', 
         'proudofjungkook', ' rodimus ', ' rpg ', 'massbuildergame', 
         ' sa ', ' cop burial ', ' hot fire test', ' isp ', ' yer ', 
         ' pic of me ', ' thanos ', 'fleshlight', 'inthemoonlight',
         ' white cat ', ' firstcontactday ', ' startrekonline ', 
         'vintage4moms', 'dblyrajsifonvhumgvo68ecys39dlcfc5e', ' dox ', 
         ' wap ', 'vampire', ' mythology ', ' purplekiss ', 'vocalist', 
         ' heck ', ' tiktok ', ' catering company ', 'in2deeprecords', 
         ' starship enterprise ', 'roddenberry', 'wonyoung', '9doodles', 
         ' yikes ', ' high end pc ', 'fleshnft', ' student au', 
         'bohemiandlive', 'hyunjujae', 'harry potter', ' yaoi ',
         ' myanmar military ', ' aint kissed ', ' meme ', ' que ', 
         ' hiphop ', ' existential crisis ', ' stanned ', ' eth ', 
         ' ig ', ' ufc ', 'insomniacgames', ' my ex', '420 dabs',
         ' infrastructure bill', ' senate ', ' nft ', 'ustedes', 
         'bossladybtarot', ' abortion ', 'theweeknd', 'geepoetry',
         ' owo ', 'unicefso', ' lana del ', 'larrytruther',
         ' championship ', 'riker', ' lyricists ', 'writeconnectllc', 
         ' reylo ', ' gap teeth ', ' the bulls ', ' vpn ', 'otelcoinc', 
         ' i fucked ', ' cyberpunk2077 ', 'weareoneexo', 'betikake', 
         'grammy', 'kawaiivibes4', 'ateez', ' exo breathes ', ' volta ',
         'digitalarts', '11x16', ' easter ', ' wizard ', ' qb ', 
         'thestuffofmemes', 'casualxtopher', ' apj ', 'smoke weed ', 
         ' trustwaller', ' emmy', 'jarirbookstore', ' snowflake ', 
         ' alien franchise ', 'douglas adams', ' a scorpio ', ' stoned ', 
         ' cunt', ' tabloid', 'esports ', ' steelers ', 'proudboy', 
         'artemisstanding', ' fisheye lens ', ' yashiro ', 'party vibe',
         ' btc ', ' prime minister ', ' shitcoin ', ' sjw ', 
         'startrek', ' acab ', ' monster ', 'boysw', ' sia ', 
         ' syllabus ','sayhisname',
         'selenaasslut', ' bts ', 'hypersexualiz', ' subs ', '4biden ',
         ' worldhealthday ', 'squishyunoh', ' coinapp ', '1bitcoinash',
         'universe is god ', ' guided meditation ', 'ship troopers', 
         'picture of the day', 'tr760006701000000076380895', 
         ' fire sign', ' pixar movie ', 'chartdata', ' contact day ',
         'cryptocurren', ' my youtube', ' sexc ', 'sunnycheungky', 
         'wildriftvideos', '4chan', 'fanboys', 'jungkookers', 
         ' hitler ', ' bebe ', ' baby boy ', 'saytheirname', 
         ' you exo ', ' brady ', ' star crossed ', 'exo gsd',
         'confomovie', ' stabeing ', 'yayabuzzgame', ' psth ', 
         ' burgerking ', 'startrekrp', ' video gam', ' safe space ', 
         ' muggle', 'au writer', '666666', 'photocontests', '80x25', 
         ' fineart ', 'senqyoun', ' kelvin timeline ', ' slayed ',
         ' gmb ', ' karma ', 'tyler the creator', 'bibliography',
         'frick ', ' asmr', ' ad ', ' exo leaving sm ', ' jets fans ',
         ' shitpost', 'cinematic universe', ' 8th house ', 'ur gay', 
         ' your handle', 'harley quinn', ' playoff', 'homeschoolfun', 
         ' bik3r boi ', ' st pa', ' april fool', 'jesus christ', 
         ' morning vibes ', 'seattlesurge', ' super bowl ', ' sith ', 
         ' industry night ', ' doink ', 'jongin','jaxdotcom', 
         ' nobody  absolutely nobody ', ' a saggitarius ', ' s n a z ',
         ' super mario ', ' brittania ', ' miley cyrus ', ' bandom ', 
         'erotica', ' wtnv ', ' dabbing dad ', ' y i k e s ', 
         ' code jam', ' first comment ', 'mansion', ' knock knock ',
         ' i just got ', ' werewolf ', 'letsnamjoon', ' cosmetology ',
         ' sim ', ' men aint shit ', ' merch ', ' wjsn ', 'nydevito',
         ' best spaghetti ', ' fics ', 'ust uploaded ', ' ratchet ', 
         'didyouknownfact', ' i just wanna ', 'seokgrls', 'fotografia', 
         ' kong ', '10th grade', ' 9gag ', 'witchesof', 'livetweet', 
         'bajenge', 'arrested', 'traytaymakeup', ' learn to spell', 
         ' detects deepfakes ', 'club   price', ' barking ', 
         'dongmincheeks', ' ppl be fucked', 'billboards', 
         ' centre for seismology ', 'face mask', 'workingfromhome',
         'rasicsmisnotcomedy', 'superframemedia', ' halsey ', 
         ' artemis au ', ' this character ', ' new lego ', 
         ' women killed', ' lucky to have each', 'artminx', 
         ' orville ', 'postmates', ' lyricist ', ' wts ',
         'loomingartemis', ' barista au', 'bluemoonstudios',
         'irishnewsbiz', 'on twitch ', 'dickpic', ' bitcoin ', 
         'teslaowners', ' birthday ', ' human au ', ' singer ', 
         ' go live ', 'horny', ' chicken ad ', ' nutted ', 
         ' au portrayal ', ' solo art', 'newlisting', ' natal ', 
         ' a virgo ', 'cherrykissjoon', ' corn syrup ', ' a leo ', 
         'fanfic', 'yerenewenjoygi', ' b0mb ', ' kink ', ' gg ', 
         'sun moon rising', ' kehlani ', ' babcock ', 'shatner', 
         ' au sketch', ' mingi ', 'hatecrimebill', ' idol ', 
         ' your download ', 'wonrobot', ' 30 seconds to mars ', 
         ' tesla stock ', ' midget ', ' april is', 'nvslive', 
         ' the devil ', ' a libra ', 'dhwani', 'jared leto', 
         ' findom ', ' comcast ', ' riverdale ', ' woman day ', 
         ' notstakedtokens ', 'familyhistoryandgenealogyseries',
         ' like tweet ', ' vagina ', 'twitchnetwork',
         ' dodge challenger ', ' yg ', ' midweekliving', ' ass ', 
         'newyorkerunion', 'treasuremembers', ' kitkat ', ' bitter ex', 
         'four seasons la', 'middaycontest', 'guinness world', 
         '  breaking  ', ' rapper ', ' sl character ', ' dawg ',
         'jam magno ', 'latenightmovie', 'couchbrewery', 
         ' car clogged la ', ' idgaf ', 'stray kids', 'k the pol',
         ' spot the difference', 'cheemsburbglar', 
         ' exos special appearance ', ' fandom ', ' soundcloud ',
         ' deffo', ' esa dog ', ' yoshi ', ' hi hi ', ' domme ',
         'bodybuilding', ' kawaii ', ' tucker carlson ', 
         'villagers in russia', 'bitches', ' sims ', ' cuck ',
         'magicschoolbus', ' iphone ', 'pixielatedesign', ' cleopatra ',
         ' warhammer', 'ex0atmospheric', 'killerklowns', ' bruno mf ', 
         ' twittersupport ', ' a gemini ', 'selfpromosaturday', 
         ' sk 8', ' puta ', 'direct message me', 'cb4yr', ' iost ',
         ' q iss ', 'enhypen', 'realestateagent', ' godzilla ', 
         ' nobody absolutely nobody ', ' my gf ', ' mlp ', ' lyrics ',
         'qosicart', ' gay space communism ', ' ootd ', 
         ' hardest jobs in the universe ', ' ive fucked ', '5sos', 
         ' hoe ', ' this footage ', ' fag ', 'pizza delivery guy', 
         ' my mother', ' insta ', 'jazzersize', 'ravenclaw', 
         ' dio ', ' tw ', 'mostrologyy', 'nodegree2dev', 
         'this drawing', ' pixel 5 ', '13b603b66', 'kpop', 
         ' clowning ', ' puns ', 'my aesthetic vibe', 
         ' ufotwitter ', 'sookramsbrewing', ' cloud space ', 
         ' exos ', ' starshipent ', ' fanmeeting ', 
         ' animal crossing ', ' primary school ', 'sexysaturday', 
         ' xiuweet ', ' album ', ' being sad is not low ', 
         'yall tl v yall twitter', 'breadfootage', ' tokyo drift ', 
         ' cis actors ', 'exols', ' beyonce ', 'germanyiniraq', 
         'sportscenter', ' kp0p', 'sungjin', ' woollim', '4teaching', 
         ' innit ', 'perv', 'cybertruck', ' ambisverse ', 
         'mobicommunitycorner', ' lego idea ', 'parkjihoon',
         'abroadinjapan', 'mysticism', 'talkradio', ' recap', 
         ' space needle ', ' trailer ', ' add me ', ' mlhc ',
         ' github daily development activity ', ' exo got ', 
         ' religion ', ' puto ', ' mmj ', 'btsieverse', ' ama ',
         'ventuskycom', ' dm ', 'toyassgirl', 'minhyu', ' pepe ', 
         'tarotcards', ' tengo ', ' chad ', ' e3 ', 'scamfake', 
         'ratedfgamingllc', 'thatmlpgamer', ' l e n g t h y ', 
         ' clowned ', 'yooyeonseokph', ' gamestop ', ' name for a cat',
         ' anris ', 'the jews', 'australian open', ' kitten content ', 
         ' garak ', ' george bush ', 'crypto', 'chaesuart', 'chonky', 
         'steven universe', 'shibas', 'spacegabpodcast', 'tranny',
         ' sailor venus ', ' song ', ' cpn ', 'baehoyuong',
         ' avrillavigne ', 'meghanandharry', ' astronaut au',
         ' uwu ', ' daddy ', ' grade student ', ' space au ',
         ' mr fantastic ', ' exo concert', ' marvel universe ', 
         ' want exo to ', 'furrson', ' a capricorn ', ' yo lo ', 
         'picard', ' antifa ', 'happybdaytome11', 'onlineexhibition', 
         'put into the universe',' variety show ',
         'ubereats', ' el catpo ', 'attackont', 'chuueoy', ' piss on ', 
         ' yo no ', ' spelling errors ', 'fortnite', 'smallbeerpress',
         'thetoychronicle', ' pendejo', ' justice league', ' romulus', 
         ' wifey ', 'achrisart', 'wwwteslagiveorg', ' freebie ',
         'drunkscribe', 'sunwoosunn', ' superintendant ', 
         ' a gang ', 'tgirl', 'ozonies', 'imbofication', ' dnd5e ', 
         'pridepublishing', ' 2017 republicans ', ' sponsor update ',
         ' murder mystery ', ' yoongi ', 'gyulmeetsworld', ' pm modi ',
         'tarot', 'british museum', 'miumiuofficial', ' stonk ', 
         'selenagomez', ' feburary is ', ' in my lap ', 
         'buyastrangerabook', 'during lockdown', 'sabesdoesart', 
         ' xbox ', 'musicmonday', ' british tabloid ', ' dios ', 
         'pseudoscience', 'ncontextmukbang', 'ringabelcaaffordable',
         'outofcontext', ' mandalorian ', ' stonks ', 'super smash',
         'japancore', 'gimped', ' abuelita ', 'live interview', 'simp',
         'thankyoucirce', ' spock ', 'voucher code', 'ilysm', ' nigg',
         'piersmorgan', 'doge', 'ghanasport', 'brunomars', 'spongebob',
         'souljaboy', 'classicstarwars', 'artisqueer', 'coinmarket',
         ' dionysus ', ' ziggy ', 'boycottutah', 'nbatopshotthis', 
         ' liminal', ' hogayi ', ' james charles ', 'taekoobots', 
         ' reddit ', ' roleplay ', 'artemisentm', ' junior school ',
         'followyourbuyer', ' this link ', ' misinformation ', 
         ' altseason ', ' our music ','emgauthor', 'bootahflytattoo',
         'jobincore', ' green run', ' f*ck', 'poofbuttmutt', 
         'whatartshealth', 'vantetaetaekook', 'indiegame', ' hey guys ', 
         ' aoc', ' sailor mars ', 'superstarwonwoo', 'freeddawg', 
         'like comment', 'killerstockcall', ' therapy dog',' god bless ',
         'complete our survey', ' gucci ', 'lostartlimited',
         ' want an easy way to get a', ' road closure ', ' anus ', 
         'nothingbutsex', ' ufc', ' dilf ', ' my esa', ' mk ', 
         ' gamergirl', ' my pup ', ' rona ', ' twizz', ' an exo ', 
         ' spacex drink', ' facebook ', '30 seconds to mars',
         'army exo', ' por ', 'gravy train', 'low low price', 
         ' erotic ', 'petenajarian', 'bert thompson', 
         ' good space mentaility', 'throw back tuesday', 
         'hotdogstand', 'test math brutal', 'exo cock', 'th house', 
         'st house', 'rd house', 'beautiful without makeup',
         'modern day pagan', 'worklifebalance', ' piano melody',
         ' kin luna ', ' hbd ', 'burfday', 'wellnesswednesday',
         'artemis troof', ' dominate cricket', 
         'rookie exo', ' en esa ', ' qui ' , 'bookmark link', 
         'taylorswift', 'hit music', 'white supremacist terror', 
         'plastic bunny hold', 'pre order', 'cmsphupdates',
         ' usagi ', 'artemis thing app', 'luna artemis', 
         'historymonth', 'codepink', 'wonder woman', 
         'fordnation', 'ice cream', 'license photo',
         'akira', 'bar support', 'torontosun', 'artemis mgmt', 
         'takatifu', 'florida insurance', 'primetime tv', 
         'freesenegal', 'deeplearn', '4th winner', 'art museum',
         'birth chart', 'junyoung', 'ultimo', 'gizbot',
         'via youtube', 'hb 2013', 'internationalspacestation passing',
         'torontoobserver', 'radio edit', ' rompe ',
         'starship take group', ' adorable video', 'video edits ', 
         'datang', 'izmminone', 'premierleague',
         'kingkong', 'etsy', 'wontonin', 'annyeongz', 'pleaseretweet',
         'fast food twitter', 'attack titan', 'wake bake',  
         'web drama', 'ted cruz', 'fuck child', '100daysof', 'banger', 
         'tv special', '5x3', 'skyrim', 'jungkook', 'lengwrites', 
         'ejaculation', 'lyric video', 'cardodging', 'brujeria',
         ' woth magic ', 'youtube son', 'paste link', 'woosan', 
         'artemis richochet', ' whipped chai ', ' swaddle art', 
         'mar entering gemini', 'javascript', 'reupload', 
         ' acoustic cover ', 'andrew elementary', 'spring grove int', 
         'dunseith public', 'student comment', 'weird pronunciation',
         'christian school', 'gaither high', 'stranship', 
         'geographyteacher', 'bandtan', 'ssg', 'icky thong', 
         'sex class', 'player coach staff', 'click photo', ' almanac', 
         'new menu', 'face post', 'astrological sign', 'trudeau', 
         'degree 2017', 'new blog', ' kid suspended ', 'marco rubio'
         ' elmer ', ' fudd ', ' impediment ', ' mawrs ', ' retard ',
         ' voiceover ', ' lisp ', ' studder ', ' accent ', 
         ' antichrist ' , ' sjw ', ' dwopping ', ' arthursday ',
         ' junior high ', ' shout out', 'shoutout', 'acadamy', 
         ' middle school ', ' high school ', ' secondary school ', 
         ' academy ', ' ibc ', 'esport', 'examcancelhoga', 
         ' dcpublicschool', ' brewsters ', ' head coach ',
         'findyourthing', 'gangsters', 'rbandme', 'music', 
         'homework',' ovni ','wandavision','dogecoin','bcu_ry', 
         'termpaper', 'newmusic','nowplaying', 'fridaylivestream',  
         'btspace', 'cravity',  'bitcoin', 'stanworld', 'izone', 
         'colchester', 'nowplaying', 'onlineclass', 'worldbookday', 
         'satyamevajayate', 'snydercut', 'mamamoo',  'eva71',
         'paralleluniversecontinues', 'dogecoin', 'showtimebet',
         'didyouknow', 'harrystyles',  'grammy', 'etsy',
         'myanmarmilitaryterrorists', 'poshmark', 'stpatricksday', 
         'mindbreeze', ' ad ', 'artoftheblue', 'essaydue', 'bigdata', 
         'aylesbury',  'piday', 'harpenden',  'soundcloud',
         ' doge ', 'zacksnyders', ' nft ',  'ifttt', 'shopee',
         'bjpmodel', 'chanyeol','baekhyun','xiuweet', 'xiumin',
         'oprahmeghanharry', 'auspixius', ' suho ', 'dollwith', 
         'icanimagine', 'thewildsspace',' dyk ', 'artemis_twt', 
         'esa_celebnews', 'superstraight', 'seekthetruth', 'izone', 
         'tarotbybronx', 'cryptoart', 'meme king', 'minecraft', 
         'artemis and luna', 'brasileiro', 'myanmar coup', 'baekhyun', 
         'doyoung', 'kpop', 'cuddles', 'bruno mars', 
         'space jam', 'bible says', 'templetonrel']

In [ ]:
regex = r"\*s(http[s]?:\S)|(\S.((com)|(org)|(gov)|(co)|(uk))\S*)\*s"
#  to update our display to see how long it's taking 
end point = len(df)
reset(df)
# go through the dataframe and really clean up the text
for i in range(len(df)):
    num = len(df)
    to_lem = df.at[i, 'text'] 
    # first we remove emojis
    to_lem = deEmojify(to_lem)
    # then urls
    to_lem = re.sub(regex, "", to_lem, flags=re.I)
    # then non-ascii characters 
    to_lem = to_lem.encode("ascii", "ignore").decode()
    # turn into a list to replace contractions 
    to_lem = to_lem.split()
    for j in range(len(to_lem)):
      if to_lem[j] in replacements:
        to_lem[j] = replacements[to_lem[j]]
    lemmed = []
    for item in to_lem:
        # remove punctuation 
        item = re.sub(r'[^\w\s]', '', item)
        # split it up again, since the contractions would be one item,
        # regardless of length 
        item = item.strip()
        if item != ' ':
          lemmed.append(lemmatizer.lemmatize(item))
    # remove stop words 
    filt_sen = [w for w in lemmed if not w in stop_words and len(w) >=2]  
    # addition I'd like to make for the future
    # filt_sen = [w for w in filt_sen if w.startswith('@') == False]
    # see if there is any innapropriate content 
    check =  any(item in filt_sen for item in list_to_drop)
    # filter to minimum sentence length 
    if len(filt_sen) >= 6 and check == False: 
      lemstr = ' '.join(filt_sen) 
      # check and correct spelling to the best we can
      textBlb = TextBlob(lemstr) 
      df.at[i, 'text'] =  textBlb.correct()
    # if it isn't long enough, we can't draw meaningful conclusions
    else:
      df.drop(index=i, inplace=True)
    # this takes hours so good to print it so you know it's really
    # working and not stuck 
    display(f"Checking text { i + 1 } of {end_point}:  ",  
            f"{(i + 1) * 10000 // (len(df)) / 100}% Processed",
            f"{num)} Remaining ")

In [ ]:
# check that it worked 
df.head(10)

In [ ]:
df.describe()

In [ ]:
# export the data 
df = df.to_csv('cleaned.csv')